# Preparing summary data for AIB bank location and reviews

Collected data using google maps for AIB bank branch locations and phone numbers in Dublin area. Removed any ATMs, head office or other types of sites that were not interacting with customers.

Used unique phone numbers to collect the average rating /5 on google reviews and the amount of reviews posted. 

This data can now be used to select branch reviews with different scores.

In [1]:
import pandas as pd #pandas for panel data
import re # regex for matching expressions to filter dataset

In [2]:
# load in scraped data for 'AIB in Dublin' google maps search

df = pd.read_csv("../data/aib_dublin.csv")

#remove any rows containing ATM data or other types site, only want banks not head office etc

df = df[~df['OSrXXb'].str.contains("ATM|Lobby|Merchant|Limited|Head|Corporate|Allied|Direct")].reset_index(drop=True)

# remove branches temporarily closed for now

df = df[~df['rllt__details 3'].str.contains("Temporarily")].reset_index(drop=True)

# drop some unnecessary columns
df.drop(columns=['UbRuwe', 'yYlJEf href', 'BSaJxc', 'yYlJEf href 2', 'rllt__details 2', 'rllt__details 3'], inplace=True)

# get rid of commas in details
df['rllt__details'] = df['rllt__details'].str.replace(',','')

# rename columns
df.rename(columns={'OSrXXb':'name', 'rllt__details':'location'}, inplace=True)

In [3]:
# the data has a 'location' column with a phone number in it. I want to extract the number to a new column to make sure there are no duplicate numbers
# the numbers begin with the symbol ·  so will match this with regex and extract after that

# Define a function to extract text after the symbol ∑
def extract_text_after_symbol(text):
    # Use regex to find and extract text after the symbol ∑
    match = re.search(r'·\s*(.*)', text)
    if match:
        return match.group(1)  # Return the captured text after the symbol
    else:
        return None  # Return None if the symbol is not found

# Create the 'number' column by applying the extract_text_after_symbol function
df['number'] = df['location'].apply(extract_text_after_symbol)

# use pattern to remove the numbers from the location columns now
df['location'] = df['location'].str.replace(r'·(.*)', '', regex=True)

# Drop all duplicates in the 'number' column except for first one
df.drop_duplicates(subset=['number'], keep='first', inplace=True)

In [4]:
# write this to csv
df.to_csv("../data/aib_banks_dublin_cleaned.csv", index=False)

In [5]:
# used the phone numbers to collect review summary data from google. Merge this with df using the unique numbers

review_summary = pd.read_csv("../data/number_reviews.csv")

df_merged = df.merge(review_summary, on = 'number')

df_merged.head()

,name,location,number,google_rating,num_reviews
0,AIB Bank,100 101 Grafton Street,(01) 671 3011,2.9,109
1,AIB Bank,7/12 Dame St,(01) 679 3211,2.9,45
2,AIB IFSC,Dublin 1,(01) 829 1880,4,1
3,AIB Bank,61 Richmond St S,(01) 478 4533,1.9,49
4,AIB Bank,126 128 Capel St,(01) 872 1022,3.3,127


In [7]:
# write to csv 

df_merged.to_csv("../data/aib_dublin_reviews_summary.csv", index=False)